In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [ ]:

INPUT_DIR = Path('/kaggle/input/playground-series-s4e7')
TARGET_NAME = 'Response'

train_data = pd.read_csv(INPUT_DIR / 'train.csv')
test_data = pd.read_csv(INPUT_DIR / 'test.csv')

# 오리지날 데이터를 이용했음
orig_data = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')

#  test set의 있는 특성위주로 작업할 생각한 듯
features = sorted(set(test_data.columns) - set(['id']))

In [ ]:

 # train set과 original set을 합친다.
# on = features가 같은 것만 뽑아는 매개변수
# filter = 같은 feature이 있다면 뒤에 따로 default is (“_x”, “_y”) 붙음 suffixes 매개변수로 변경 가능
# reset_index() - index 열의 생성여부를 정할 수 있음
  (train_data
    .merge(orig_data, on=features)
    .filter(['Response_x', 'Response_y'])
    .value_counts()
    .reset_index()
)

,Response_x,Response_y,count
0,0,1,4124
1,1,0,4025


In [ ]:
# assign - DataFrame에 새 열을 할당하는 메서드
#  ㄴ overide라는 column이 생김
# np.where - 조건 에 따라 x 또는 y 에서 선택된 요소를 반환
# lambda의 x는 merge된 상태의 DataFrame
# agg - 여러개의 함수를 여러 열에 적용
override_sub = (test_data
    .merge(orig_data.drop(columns=['id']), on=features)
    .assign(override = lambda x: np.where(x['Response'] == 0, 1, 0))
    .filter(['id', 'override'])
    # we can get duplicates...
    .groupby(['id'], as_index=False)
    .agg(override = ('override', 'mean')))

In [ ]:
# 원래 데이터와 일치하는 모든 행을 재정의합니다:
# 다른 노트북의 csv데이터를 가져옴
ivans_sub = pd.read_csv('/kaggle/input/s04e07-insurance-cross-selling-catboost/sub_catboost_0.89487.csv')

# how - "outer"은 합집합

(ivans_sub
    .merge(override_sub, how='outer')
    .assign(Response = lambda x: np.where(x['override'].isna(), x['Response'], x['override']))
    .filter(['id', 'Response'])
    .to_csv('override_submission.csv', index=False))

In [ ]:
# 다른 노트북의 csv데이터를 가져옴
ivans_sub2 = pd.read_csv('/kaggle/input/ps4e7-classification-generalization/submission.csv')

(ivans_sub2
    .merge(override_sub, how='outer')
    .assign(Response = lambda x: np.where(x['override'].isna(), x['Response'], x['override']))
    .filter(['id', 'Response'])
    .to_csv('override_submission2.csv', index=False))

In [ ]:
targets=["Response"]
target=targets[0]

함수 설명
* 매개변수
    * sub_list: 예측 결과를 담고 있는 DataFrame 객체들의 리스트입니다. 각 DataFrame에는 예측된 결과 값이 포함되어 있습니다.
    * cols: 평균을 계산할 때 사용할 열(column)들의 리스트입니다.
    * mean: 평균을 계산하는 방법을 선택하는 매개변수입니다. 기본값은 "AM"이며, 다음 세 가지 옵션이 가능합니다:
* "AM": 산술 평균 (Arithmetic Mean)
* "GM": 기하 평균 (Geometric Mean)
* "HM": 조화 평균 (Harmonic Mean)
* 반환값
    * sub_out: 평균 계산 후의 결과를 담고 있는 DataFrame 객체입니다. sub_list 중 첫 번째  DataFrame을 복사한 후, 지정된 평균 방법에 따라 선택한 열의 값을 수정합니다.

In [ ]:
def ensemble_mean(sub_list,cols, mean="AM"):

    """
    이 함수는 특정 결과가 포함된 결과 목록이 주어지면 산술 평균/측량 평균/하모닉 평균을 계산합니다.
    """

    sub_out=sub_list[0].copy()
    if mean=="AM":
        for col in cols:
            sub_out[col]=sum(df[col] for df in sub_list)/len(sub_list)
    elif mean=="GM":
        for df in sub_list[1:]:
            for col in cols:
                sub_out[col]*=df[col]
        for col in cols:
            sub_out[col]=(sub_out[col])**(1/len(sub_list))
    elif mean=="HM":
        for col in cols:
            sub_out[col]=len(sub_list)/sum(1/df[col] for df in sub_list)

    return sub_out

In [ ]:
# 전처리 메소드 함수화
from sklearn.preprocessing import MinMaxScaler
def scale(df,targets):
    sc=MinMaxScaler()
    df[targets]=sc.fit_transform(df[targets])

    return df

sub1=scale(ivans_sub,targets)
sub2=scale(ivans_sub2,targets)



```
sub_list=[sub1, sub2]
weights=np.square([6, 3])

```

* sub_list: 예측 결과가 담긴 두 개의 데이터프레임, sub1과 sub2.
* weights: 각 데이터프레임에 적용할 가중치. [6, 3]의 제곱을 취해 [36, 9]로 변환됩니다.

---




```
if len(sub_list)==len(weights):
    weighted_list = [item for sublist, weight in zip(sub_list, weights) for item in [sublist] * weight]

```
조건문은 sub_list와 weights의 길이가 같은지 확인합니다.<br>
zip(sub_list, weights)는 sub_list와 weights를 쌍으로 묶습니다.<br>
리스트 내포를 통해 각 sublist를 weight만큼 반복하여 weighted_list를 만듭니다. <br>
즉, sub1은 36번, sub2는 9번 복사되어 weighted_list에 추가됩니다. <br>
이 과정은 가중치를 반영하여 데이터프레임을 중복시키는 방식으로 가중치를 적용하는 것입니다.

---




```
sub_ensemble = ensemble_mean(weighted_list, targets, mean="AM")
sub_ensemble.to_csv('submission.csv', index=False)
sub_ensemble.head()

```

* ensemble_mean 함수를 사용하여 weighted_list의 가중치가 반영된 데이터프레임에 대해 산술 평균(AM)을 계산합니다.
* 계산된 결과는 sub_ensemble에 저장됩니다.
* sub_ensemble을 'submission.csv' 파일로 저장합니다. index=False는 데이터프레임의 인덱스를 파일에 포함하지 않도록 설정합니다.
* sub_ensemble.head()는 최종 결과 데이터프레임의 상위 5개 행을 출력합니다.


---


In [ ]:
sub_list=[sub1,sub2]

# 각 모델의 예측 결과에 부여할 중요도를 weights 안에 변수로 지정
weights=np.square([6,3])
if len(sub_list)==len(weights):
    weighted_list = [item for sublist, weight in zip(sub_list, weights) for item in [sublist] * weight]

sub_ensemble=ensemble_mean(weighted_list,targets,mean="AM")
sub_ensemble.to_csv('submission.csv',index=False)
sub_ensemble.head()

,id,Response
0,11504798,0.004479
1,11504799,0.673191
2,11504800,0.242349
3,11504801,0.000162
4,11504802,0.214106


np.random.beta는 베타 분포에서 무작위 샘플을 생성하는 데 사용됩니다. 베타 분포는 [0, 1] 구간에서 정의되는 연속 확률 분포로, 주로 베이지안 통계학에서 확률을 모델링할 때 자주 사용됩니다.

In [ ]:
# 랜덤 값으로 임의의 확률을 지정하는 코드인데... 왜 넣었는지 모르겠음
sub_ensemble['Response'] = np.random.beta(1, 1, size=len(ivans_sub))
sub_ensemble.to_csv('submission.csv', index=False)